# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import re
import nltk
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///mydf.db')
df = pd.read_sql_table('mydf',engine)
X = df['message']
y = df.iloc[:,4:]
print(X,y)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def my_pipe():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = []
model = my_pipe()
predict_pipe=model.fit(X_train, y_train)

CPU times: user 1min 18s, sys: 285 ms, total: 1min 18s
Wall time: 1min 18s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
%%time
predict_y_test = predict_pipe.predict(X_test)

#Using Classification_report to find precision, recasll, f1-score and support
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in predict_y_test]), target_names = df.columns[4:]))

                        precision    recall  f1-score   support

               related       0.81      0.38      0.52      1038
               request       0.00      0.00      0.00        27
                 offer       0.75      0.50      0.60      2648
           aid_related       0.53      0.08      0.14       504
          medical_help       0.73      0.07      0.13       317
      medical_products       1.00      0.03      0.06       167
     search_and_rescue       0.00      0.00      0.00       116
              security       0.65      0.09      0.16       225
              military       0.00      0.00      0.00         0
           child_alone       0.83      0.21      0.34       398
                 water       0.88      0.28      0.43       710
                  food       0.82      0.20      0.32       561
               shelter       0.80      0.04      0.08        99
              clothing       0.50      0.01      0.01       144
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [8]:
%%time
parameters = {
    'vect__ngram_range': ((1, 2), (2, 2)),
    'vect__max_df': (0.75, 1.0),
    'clf__estimator__n_estimators':[10, 15]
}

cv = GridSearchCV(model, param_grid = parameters)

CPU times: user 167 µs, sys: 0 ns, total: 167 µs
Wall time: 173 µs


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
#Warning: This took 52 Minutes!!
# Best Parameters: {'clf__estimator__n_estimators': 10, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
%%time
cv.fit(X_train, y_train)
print('Best Parameters:', cv.best_params_)

Best Parameters: {'clf__estimator__n_estimators': 10, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
CPU times: user 51min 55s, sys: 1.5 s, total: 51min 56s
Wall time: 52min 19s


In [12]:
y_test_pred = cv.predict(X_test)
    
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_test_pred]), target_names = df.columns[4:]))

                        precision    recall  f1-score   support

               related       0.83      0.42      0.56      1038
               request       0.00      0.00      0.00        27
                 offer       0.75      0.47      0.57      2648
           aid_related       0.57      0.06      0.11       504
          medical_help       0.66      0.09      0.15       317
      medical_products       0.88      0.04      0.08       167
     search_and_rescue       0.00      0.00      0.00       116
              security       0.64      0.09      0.16       225
              military       0.00      0.00      0.00         0
           child_alone       0.84      0.29      0.44       398
                 water       0.84      0.47      0.60       710
                  food       0.78      0.26      0.39       561
               shelter       1.00      0.04      0.08        99
              clothing       0.67      0.01      0.03       144
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
#Adding new Feature!

#Class StartVerbExtractor added to use it as a feature in "my_new_pipe"
class StartVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

def my_new_pipe():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('first_verb', StartVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

In [20]:
%%time
model = []
model_new = my_new_pipe()
improved_pipe=model_new.fit(X_train, y_train)

CPU times: user 2min, sys: 687 ms, total: 2min
Wall time: 2min 1s


In [21]:
%%time
predict_y_test_new = improved_pipe.predict(X_test)

#Using Classification_report to find precision, recasll, f1-score and support
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in predict_y_test_new]), target_names = df.columns[4:]))


                        precision    recall  f1-score   support

               related       0.80      0.35      0.49      1038
               request       0.00      0.00      0.00        27
                 offer       0.75      0.52      0.62      2648
           aid_related       0.57      0.09      0.15       504
          medical_help       0.62      0.07      0.13       317
      medical_products       1.00      0.04      0.07       167
     search_and_rescue       0.00      0.00      0.00       116
              security       0.50      0.04      0.07       225
              military       0.00      0.00      0.00         0
           child_alone       0.85      0.20      0.32       398
                 water       0.87      0.30      0.45       710
                  food       0.80      0.25      0.38       561
               shelter       0.77      0.10      0.18        99
              clothing       0.00      0.00      0.00       144
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
model_new.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__first_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipe

In [ ]:
%%time
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.75, 1.0),
        'clf__estimator__n_estimators':[10, 15],
        'features__text_pipeline__tfidf__use_idf': (True, False)
    }

pipeline_cv = GridSearchCV(model_new, param_grid = parameters)
pipeline_cv.fit(X_train, y_train)
print('Best Parameters:', pipeline_cv.best_params_)

In [ ]:
Y_pred = pipeline_cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = category_names))

### 9. Export your model as a pickle file

In [ ]:
filename = 'my_model.sav'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.